In [1]:
%pip install --upgrade pip 
%pip install transformers torch torchvision 
%pip install devinterp
!source llc/bin/activate

import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu" 
# if you want to use your TPU (note that torch_xla is not by default installed, install using pip install devinterp[tpu]) 
# import os 
# os.environ["USE_TPU_BACKEND"] = "1" 
# import torch_xla.core.xla_model as xm 
# DEVICE = xm.xla_device()

import warnings 
import torch 
import torchvision 
from torch.nn import functional as F 
from transformers import AutoModelForImageClassification 
from devinterp.optim import SGLD 
from devinterp.slt.sampler import estimate_learning_coeff_with_summary 
from devinterp.utils import plot_trace, default_nbeta 

warnings.filterwarnings("ignore")

/Users/swey/.zshenv:.:1: no such file or directory: /Users/swey/.cargo/env
Note: you may need to restart the kernel to use updated packages.
/Users/swey/.zshenv:.:1: no such file or directory: /Users/swey/.cargo/env
Note: you may need to restart the kernel to use updated packages.
/Users/swey/.zshenv:.:1: no such file or directory: /Users/swey/.cargo/env
Note: you may need to restart the kernel to use updated packages.
/Users/swey/.zshenv:.:1: no such file or directory: /Users/swey/.cargo/env
zsh:source:1: no such file or directory: llc/bin/activate


In [2]:
%cd tracr 
%pip install .
%cd ..

/Users/swey/Documents/Courses/CMSC476/llc-interpretability/tracr
/Users/swey/.zshenv:.:1: no such file or directory: /Users/swey/.cargo/env
Processing /Users/swey/Documents/Courses/CMSC476/llc-interpretability/tracr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tracr: filename=tracr-1.0.0-py3-none-any.whl size=114088 sha256=9f2ed8e81de499f1abde4b1dc85ae0d24285a6f6e9518285b8564d5f21a98fed
  Stored in directory: /private/var/folders/dx/kz2_2hrs76d4rj5nj23j1fmr0000gn/T/pip-ephem-wheel-cache-x0jrnk39/wheels/51/bf/b5/d63e19001d6364acdc600c234ac7b55e31757f2be439f88f1b
Successfully built tracr
  Attempting uninstall: tracr
    Found existing installation: tracr 1.0.0
    Uninstalling tracr-1.0.0:
      Successfully uninstalled tracr-1.0.0
Note: you may need to restart the kernel to use updated packages.
/Users/swey/Documents/Courses/CMSC476/llc-interpretability


In [16]:
import torch
import torch.nn.functional as F
from devinterp.slt.sampler import estimate_learning_coeff_with_summary
from devinterp.optim import SGLD
from devinterp.utils import default_nbeta
from peak import get_peak_model
from tracr.haiku_to_pytorch import haiku_to_pytorch
from dataloaders import makePeakDataLoader

class TracrWrapper(torch.nn.Module):
    def __init__(self, tracr_pytorch_model):
        super().__init__()
        self.tracr_pytorch = tracr_pytorch_model
        for p in self.tracr_pytorch.parameters():
            p.requires_grad = True
        
    def forward(self, input_seqs):
        outputs = []
        for seq in input_seqs:
            result = self.tracr_pytorch.model.apply(seq)
            decoded = result.decoded[1:]
            outputs.append(torch.tensor([float(x) for x in decoded], dtype=torch.float32))
        output = torch.stack(outputs)
        param_sum = sum(p.sum() * 0.0 for p in self.tracr_pytorch.parameters())
        return output + param_sum

def evaluate(model, batch):
    inputs, targets = batch
    outputs = model(inputs)
    return F.mse_loss(outputs, targets.float()), {"outputs": outputs}

init_model = get_peak_model()
tracr_pytorch = haiku_to_pytorch(init_model)
model = TracrWrapper(tracr_pytorch)

loader = makePeakDataLoader()

# test gradient flow
batch = next(iter(loader))
loss, _ = evaluate(model, batch)
for p in model.parameters():
    if p.grad is not None:
        p.grad.zero_()
loss.backward()
params_with_grad = sum(1 for p in model.parameters() if p.grad is not None and p.grad.abs().sum() > 0)

learning_coeff_stats = estimate_learning_coeff_with_summary(
        model,
        loader=loader,
        evaluate=evaluate,
        sampling_method=SGLD,
        optimizer_kwargs=dict(lr=1e-5, localization=1.0, nbeta=default_nbeta(loader)),
        num_chains=3,
        num_draws=100,
        device='cpu',
        online=True,
)

if 'learning_coeff_stats' in globals():
    llc_trace = learning_coeff_stats.get("llc/trace", None)
    if llc_trace is not None:
        if isinstance(llc_trace, torch.Tensor):
            llc_trace = llc_trace.detach().cpu().numpy()
        else:
            try:
                llc_trace = torch.tensor(llc_trace).numpy()
            except Exception:
                pass
        print("LLC trace values:")
        print(llc_trace)

Chain 2: 100%|██████████| 100/100 [00:03<00:00, 32.36it/s]

LLC trace values:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
